In [18]:
import numpy as np
import matplotlib.pyplot as plt
import stim
from lib.stabilizer import measurement_gadgets, StabilizerCode, stabilizer_circuits
from lib.color_compass import Lattice2D, compass_to_surface
from lib.decoder import checkmatrix,pL_from_checkmatrix
from lib.stim2pymatching import estimate_pL_noisy_graph
import stimcirq
from typing import *
from cirq.contrib.svg import SVGCircuit
import pymatching

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
# Construction of a Pauli noise model

class PauliNoiseModel():
    """
    Constructs noisy Stim circuits from 2D Compass Code stabilizers and logical observables

    A noise model is defined as mapping a perfect operation to an imperfect operation
    """
    def __init__(self, one_qb_gate_rates : List[float] = [0] * 3, two_qb_gate_rates : List[float] = [0] * 15, meas_error_rate : float = 0):
        self.one_qb_gate_rates = one_qb_gate_rates
        assert(len(one_qb_gate_rates) == 3)
        self.two_qb_gate_rates = two_qb_gate_rates
        assert(len(two_qb_gate_rates) == 15)
        self.meas_error_rate = meas_error_rate

    def one_qb_pauli_noise(self) -> str:
        """ 
        Returns a string representing a single qubit Stim Pauli error channel
        """
        channel_str = 'PAULI_CHANNEL_1({},{},{})'.format(*self.one_qb_gate_rates)
        return channel_str 
    
    def two_qb_pauli_noise(self) -> str:
        """ 
        Returns a string representing a two qubit Stim Pauli error channel
        """
        channel_str = 'PAULI_CHANNEL_2({},{},{},{},{},{},{},{},{},{},{},{},{},{},{})'.format(*self.two_qb_gate_rates)
        return channel_str 
    
    def measurement_gadget(self, pauli_observable : str):
        """ 
        Stim gadget to directly measure the specified 'pauli_observable'
        """
        meas_circ = ''
        x_meas_pos = ''
        y_meas_pos = '' 
        z_meas_pos = ''
        pos = {'I' : [], 'X' : [], 'Y' : [], 'Z' : []}
        for i, pauli in enumerate(pauli_observable):
            if pauli == 'X':
                x_meas_pos += f' {i}'
            elif pauli == 'Y':
                y_meas_pos += f' {i}'
            elif pauli == 'Z':
                z_meas_pos += f' {i}'
            else:
                pass 

        meas_circ += f'MX({self.meas_error_rate})' + (x_meas_pos * (len(x_meas_pos) != 0)) + '\n' + f'MY({self.meas_error_rate})' + (y_meas_pos * (len(y_meas_pos) != 0)) + '\n' + f'MZ({self.meas_error_rate})' + (z_meas_pos * (len(z_meas_pos) != 0)) + '\n'
        return stim.Circuit(meas_circ)
    
    def stabilizer_gadget(self, stabilizer_in : str, ancilla_index : int, construction : str = 'cnot'):
        """
        Input:
            stabilizer: a single stabilizer written in terms of {I/_,X,Y,Z}
            construction: direct or hadamard:
                1) `cnot` using only CNOTs from data to ancilla along with single qubit gates
                    - H then S    : rotates Z basis -> Y basis
                    - S_dag then H: rotates Y basis -> Z basis
                    verifiable via checking that: Y stabilizer == kron(S@H,I) @ CNOT @ kron(H@S_dag)
                2) `hadamard` using H gates on ancilla and C-Pauli from ancilla to data
        Output:
            Measurement gadget

        the data qubits that are indicated in the stabilizer appear first
        the ancilla index starts from 0, which is the first ancilla qubit after the data 
        """
         # allow both '_' and 'I' in stabilizers
        stabilizer = stabilizer_in.replace('_','I')
        
        N = len(stabilizer)
        circ_string = ''
        if construction == 'cnot':
            for i, pauli in enumerate(stabilizer):
                if pauli == 'Z':
                    # Z-gates are just cnots from data to ancilla
                    noise_string = self.two_qb_pauli_noise() + f' {i} {N+ancilla_index}\n'
                    circ_string += f'CX {i} {ancilla_index+N} \n' 
                    circ_string += noise_string
                elif pauli == 'X':
                    # X-gates are conjugated by hadamards
                    noise_string_1qb = self.one_qb_pauli_noise() + f' {i}\n'
                    noise_string_2qb = self.two_qb_pauli_noise() + f' {i} {N+ancilla_index}\n'
                    circ_string += f'H {i} \n'
                    circ_string += noise_string_1qb
                    circ_string += f'CX {i} {ancilla_index+N} \n' 
                    circ_string += noise_string_2qb
                    circ_string += f'H {i} \n'
                    circ_string += noise_string_1qb
                elif pauli == 'Y':
                    # Y-gates are conjugated by S-gates and hadamards
                    noise_string_1qb = self.one_qb_pauli_noise() + f' {i}\n'
                    noise_string_2qb = self.two_qb_pauli_noise() + f' {i} {N+ancilla_index}\n'
                    circ_string = f'S_DAG {i} \n'
                    circ_string += noise_string_1qb
                    circ_string += f'H {i} \n'
                    circ_string += noise_string_1qb
                    circ_string += f'CX {i} {ancilla_index+N} \n' 
                    circ_string += noise_string_2qb
                    circ_string += f'H {i} \n'
                    circ_string += noise_string_1qb
                    circ_string += f'S {i} \n'
                    circ_string += noise_string_1qb 

            # noisy ancilla measurement
            circ_string += f'MR({self.meas_error_rate}) {N+ancilla_index}\n'
        return stim.Circuit(circ_string)


    def stabilizer_gadget_v2(self, stabilizer_in : int):
        """
        Use Stim's built in 'MPP' function
        (IS THIS PREFERRED OVER SPLITTING UP MEASUREMENTS INTO CONSTITUENT PARTS AND APPLYING CIRCUIT-LEVEL NOISE?)
        """
        # allow both '_' and 'I' in stabilizers
        stabilizer = stabilizer_in.replace('_','I')
        
        N = len(stabilizer)
        circ_string = f'MPP({self.meas_error_rate}) '
        for i, pauli in enumerate(stabilizer):
            if (pauli != 'I'):
                circ_string += f'{pauli}{i}*'
        circ_string = circ_string[:-1] + '\n'
        return stim.Circuit(circ_string)

In [20]:
def sample_low_weight_pauli(n_qubits: int, pauli_probs: List):
    """ 
    Sample a random Pauli that represents the Pauli error that occurs at each timestep

    Params:
    * n_qubits - Number of qubits in lattice
    * prob - List of probabilities of single-qubit Paulis
    """
   
    

def compile_compass_circuit(compass_code : Lattice2D, pauli_noise_model : PauliNoiseModel, noiseless_model : PauliNoiseModel, rounds : int, model_type : str):
    """ 
    We compile a compass code lattice into stim circuits with detectors between subsequent stabilizer measurements

    Params:
    * compass_code - Instance of 'Lattice2D' class that defines compass code
    * pauli_noise_model - Instance of 'PauliNoiseModel' that defines Pauli noise model
    * noiseless_model - Instance of 'PauliNoiseModel' that defines a noiseless Pauli noise model
    * rounds - Number of rounds of stabilizer measurements we look to perform
    * model_type:
        - CC --> Code Capacity
        - PH --> Phenomenological
        - CL --> Circuit-Level
    """
    compass_circuit = stim.Circuit()
    if (model_type == 'CC'):
        encoding_circ = StabilizerCode(compass_code.getS()).encoding_circuit(stim=True)
        compass_circuit += encoding_circ 

        # Perform noiseless stabilizer measurements
        for idx, sx in enumerate(compass_code.getSx()):
            compass_circuit += noiseless_model.stabilizer_gadget(sx, idx)

        for idz, sz in enumerate(compass_code.getSz()):
            compass_circuit += noiseless_model.stabilizer_gadget(sz, idz)

        return compass_circuit
    else:
        # Perform encoding into logical all-zeros state
        encoding_circ = StabilizerCode(compass_code.getS()).encoding_circuit(stim=True)
        compass_circuit += encoding_circ

        # Add dummy measurements at start of circuit (X stabs)
        num_X_stabs = len(compass_code.getSx())
        num_Z_stabs = len(compass_code.getSz())

        for idx, sx in enumerate(compass_code.getSx()):
            compass_circuit += noiseless_model.stabilizer_gadget(sx, idx)

        for idz, sz in enumerate(compass_code.getSz()):
            compass_circuit += noiseless_model.stabilizer_gadget(sz, idz)

        # Perform n rounds of stabilizer measurements and add detector
        for n in range(rounds):
            if (n > rounds - 1):
                for idx, sx in enumerate(compass_code.getSx()):
                    compass_circuit += noiseless_model.stabilizer_gadget(sx, idx)
                for idz, sz in enumerate(compass_code.getSz()):
                    compass_circuit += noiseless_model.stabilizer_gadget(sz, idz)
                for idx, sx in enumerate(compass_code.getSx()):
                    compass_circuit += stim.Circuit(f"DETECTOR({idx}, 0, {n + 1}) rec[{-1 - idx}] rec[{-1 - num_X_stabs - num_Z_stabs - idx}]")
                for idz, sz in enumerate(compass_code.getSz()):
                    compass_circuit += stim.Circuit(f"DETECTOR({idz}, 1, {n + 1}) rec[{-1 - idz}] rec[{-1 - num_Z_stabs - num_X_stabs - idz}]")
            else:
                for idx, sx in enumerate(compass_code.getSx()):
                    compass_circuit += pauli_noise_model.stabilizer_gadget(sx, idx)
                for idz, sz in enumerate(compass_code.getSz()):
                    compass_circuit += pauli_noise_model.stabilizer_gadget(sz, idz)
                for idx, sx in enumerate(compass_code.getSx()):
                    compass_circuit += stim.Circuit(f"DETECTOR({idx}, 0, {n + 1}) rec[{-1 - idx}] rec[{-1 - num_X_stabs - num_Z_stabs - idx}]")
                for idz, sz in enumerate(compass_code.getSz()):
                    compass_circuit += stim.Circuit(f"DETECTOR({idz}, 1, {n + 1}) rec[{-1 - idz}] rec[{-1 - num_Z_stabs - num_X_stabs - idz}]")
    
    return compass_circuit 

In [21]:
# Randomly select a surface density code
dim = 4
lat = Lattice2D(dim, dim)

coloring = np.random.randint(-1, 2, size=(dim-1)**2)
lat.color_lattice(coloring)
print(lat)

000---001---002---003
 |  ░  |  ░  |  ▓  |
004---005---006---007
 |  ░  |  ▓  |  ░  |
008---009---010---011
 |  ░  |     |     |
012---013---014---015



In [22]:
# Fix a Pauli noise model
one_qb_rates = [0.0] * 3
two_qb_rates = [0.0] * 15
meas_rates = 0.01
pnm = PauliNoiseModel(one_qb_gate_rates=one_qb_rates, meas_error_rate=meas_rates, two_qb_gate_rates=two_qb_rates)

In [23]:
# Define a noiseless model
one_qb_rates = [0.0] * 3
meas_rates = 0.0
noiseless_nm = PauliNoiseModel(one_qb_gate_rates=one_qb_rates, meas_error_rate=meas_rates)

### Code-Capacity Model

In [24]:
# Construct the encoding and measurement circuit for our chosen code
num_stab_meas_rounds = 2
circ = compile_compass_circuit(lat, pnm, noiseless_nm, num_stab_meas_rounds, model_type='CC')
print(circ)

H 8 1 3 4 6
TICK
CX 8 9 1 0 4 5
TICK
CX 8 10 1 2
TICK
CX 8 11 1 7 4 10
TICK
CX 8 12 3 7 4 11
TICK
CX 8 13 1 12 6 7
TICK
CX 8 14 1 13 4 12 6 10
TICK
CX 8 15 1 14 4 13 6 11
TICK
CX 1 15 4 14
TICK
CX 4 15
TICK
H 8
PAULI_CHANNEL_1(0, 0, 0) 8
CX 8 16
PAULI_CHANNEL_2(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0) 8 16
H 8
PAULI_CHANNEL_1(0, 0, 0) 8
H 9
PAULI_CHANNEL_1(0, 0, 0) 9
CX 9 16
PAULI_CHANNEL_2(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0) 9 16
H 9
PAULI_CHANNEL_1(0, 0, 0) 9
H 10
PAULI_CHANNEL_1(0, 0, 0) 10
CX 10 16
PAULI_CHANNEL_2(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0) 10 16
H 10
PAULI_CHANNEL_1(0, 0, 0) 10
H 11
PAULI_CHANNEL_1(0, 0, 0) 11
CX 11 16
PAULI_CHANNEL_2(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0) 11 16
H 11
PAULI_CHANNEL_1(0, 0, 0) 11
H 12
PAULI_CHANNEL_1(0, 0, 0) 12
CX 12 16
PAULI_CHANNEL_2(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0) 12 16
H 12
PAULI_CHANNEL_1(0, 0, 0) 12
H 13
PAULI_CHANNEL_1(0, 0, 0) 13
CX 13 16
PAULI_CHANNEL_2(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0